In [72]:
import numpy as np
import cv2
import mediapipe as mp
import pandas as pd
import os
import math

In [73]:
mp_face_mes = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
face_mesh = mp_face_mes.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [74]:
images_wide = os.listdir('eye_wide') #Warning: in all images, BOTH eyes must be wide open
images_open = os.listdir('eye_open') #Warning: in all images, BOTH eyes must be open
images_narrowed = os.listdir('eye_narrowed') #Warning: in all images, BOTH eyes must be narrowed
images_closed = os.listdir('eye_closed') #Warning: in all images, BOTH eyes must be closed

print(len(images_wide), len(images_open), len(images_narrowed), len(images_closed))

909 1461 1448 1263


In [75]:
def calculate_roll_angle(landmarks):
    left_ear_x = landmarks[234].x
    left_ear_y = landmarks[234].y
    right_ear_x = landmarks[454].x
    right_ear_y = landmarks[454].y
    angle_rad = math.atan2(right_ear_y - left_ear_y, right_ear_x - left_ear_x)
    angle_deg = math.degrees(angle_rad)
    return angle_deg

def compensate_head_roll(frame, results_mesh, W, H):
    roll_angle_deg = calculate_roll_angle(results_mesh.multi_face_landmarks[0].landmark)
    M = cv2.getRotationMatrix2D((W/2, H/2), roll_angle_deg, 1)
    frame = cv2.warpAffine(frame, M, (W, H))
    return frame

In [76]:
def update_mesh_points(image):
    try:
        frame = image.copy()
    except AttributeError:
        return None
    H, W, _ = frame.shape
    results_mesh_local = face_mesh.process(frame)
    if results_mesh_local.multi_face_landmarks:
        frame = compensate_head_roll(frame, results_mesh_local, W, H)
        results_mesh = face_mesh.process(frame)
        if results_mesh.multi_face_landmarks:
            mesh_points = np.array([np.multiply([p.x, p.y], [W, H]).astype(int) for p in results_mesh.multi_face_landmarks[0].landmark])
            for point in mesh_points:
                cv2.circle(frame, tuple(point), 1, (0, 255, 0), -1)
            return mesh_points
    return None

In [77]:
RIGHT_EYE = [362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398, 362]
LEFT_EYE = [33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161, 246, 33]

def calculate_sclera_area(mesh_points):
    lex1, lex2 = mesh_points[33, 0], mesh_points[133, 0]
    rex1, rex2 = mesh_points[362, 0], mesh_points[263, 0]
    area_sclera_left = 0
    area_sclera_right = 0
    for i in range(len(LEFT_EYE) - 1):
        area_sclera_left += (mesh_points[LEFT_EYE[i], 0] * mesh_points[LEFT_EYE[i + 1], 1]) - (mesh_points[LEFT_EYE[i + 1], 0] * mesh_points[LEFT_EYE[i], 1])
    for i in range(len(RIGHT_EYE) - 1):
        area_sclera_right += (mesh_points[RIGHT_EYE[i], 0] * mesh_points[RIGHT_EYE[i + 1], 1]) - (mesh_points[RIGHT_EYE[i + 1], 0] * mesh_points[RIGHT_EYE[i], 1])
    area_sclera_left = abs(area_sclera_left / (2*(abs(lex1 - lex2)**2)))
    area_sclera_right = abs(area_sclera_right / (2*(abs(rex1 - rex2)**2)))
    return area_sclera_left, area_sclera_right

def calculate_width_over_height(mesh_points):
    lex1 = mesh_points[33][0]
    lex2 = mesh_points[133][0]
    rex1 = mesh_points[362][0]
    rex2 = mesh_points[263][0]
    ley1 = mesh_points[159][1]
    ley2 = mesh_points[145][1]
    rey1 = mesh_points[386][1]
    rey2 = mesh_points[374][1]
    reason_left = abs(lex1-lex2)/abs(ley1-ley2)
    reason_right = abs(rex1-rex2)/abs(rey1-rey2)
    if reason_left > 20: reason_left = 20
    if reason_right > 20: reason_right = 20
    return reason_left, reason_right

def get_features(frame):
    mesh_points = update_mesh_points(frame)
    if mesh_points is None:
        return None
    area_sclera_left, area_sclera_right = calculate_sclera_area(mesh_points)
    reason_left, reason_right = calculate_width_over_height(mesh_points)
    return (area_sclera_left, area_sclera_right, reason_left, reason_right)

In [78]:
def resize_if_too_small(photo):
    if photo.shape[0] < 300 and photo.shape[1] < 300:
        photo = cv2.resize(photo, (300, 300), interpolation = cv2.INTER_CUBIC)
    return photo

In [79]:
results_wide = []
results_open = []
results_narrowed = []
results_closed = []

print('Processing wide images...')
for image in images_wide:
    frame = cv2.imread('eye_wide/'+image)
    if frame is None:
        continue
    frame = resize_if_too_small(frame)
    result = get_features(frame)
    if result:
        results_wide.append((result[0], result[2]))
        results_wide.append((result[1], result[3]))

print('Processing open images...')
for image in images_open:
    frame = cv2.imread('eye_open/'+image)
    if frame is None:
        continue
    frame = resize_if_too_small(frame)
    result = get_features(frame)
    if result:
        results_open.append((result[0], result[2]))
        results_open.append((result[1], result[3]))

print('Processing narrowed images...')
for image in images_narrowed:
    frame = cv2.imread('eye_narrowed/'+image)
    if frame is None:
        continue
    frame = resize_if_too_small(frame)
    result = get_features(frame)
    if result:
        results_narrowed.append((result[0], result[2]))
        results_narrowed.append((result[1], result[3]))

print('Processing closed images...')
for image in images_closed:
    frame = cv2.imread('eye_closed/'+image)
    if frame is None:
        continue
    frame = resize_if_too_small(frame)
    result = get_features(frame)
    if result:
        results_closed.append((result[0], result[2]))
        results_closed.append((result[1], result[3]))

Processing wide images...


c:\Users\felip\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Processing open images...
Processing narrowed images...


C:\Users\felip\AppData\Local\Temp\ipykernel_16720\1014457737.py:27: RuntimeWarning: divide by zero encountered in long_scalars
  reason_right = abs(rex1-rex2)/abs(rey1-rey2)
C:\Users\felip\AppData\Local\Temp\ipykernel_16720\1014457737.py:26: RuntimeWarning: divide by zero encountered in long_scalars
  reason_left = abs(lex1-lex2)/abs(ley1-ley2)


Processing closed images...


In [80]:
print(len(results_wide), len(results_open), len(results_narrowed), len(results_closed))

1616 2734 1892 2472


In [81]:
results_wide = np.array(results_wide)
results_open = np.array(results_open)
results_narrowed = np.array(results_narrowed)
results_closed = np.array(results_closed)

In [82]:
df = pd.DataFrame(columns=['normalized_sclera_area', 'width_over_height', 'label'])

for value in results_wide:
    df.loc[len(df)] = {'normalized_sclera_area': value[0], 'width_over_height': value[1], 'label': 'wide'}
for value in results_open:
    df.loc[len(df)] = {'normalized_sclera_area': value[0], 'width_over_height': value[1], 'label': 'open'}
for value in results_narrowed:
    df.loc[len(df)] = {'normalized_sclera_area': value[0], 'width_over_height': value[1], 'label': 'narrowed'}
for value in results_closed:
    df.loc[len(df)] = {'normalized_sclera_area': value[0], 'width_over_height': value[1], 'label': 'closed'}

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8714 entries, 0 to 8713
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   normalized_sclera_area  8714 non-null   float64
 1   width_over_height       8714 non-null   float64
 2   label                   8714 non-null   object 
dtypes: float64(2), object(1)
memory usage: 272.3+ KB


In [83]:
df

,normalized_sclera_area,width_over_height,label
0,0.341435,2.000000,wide
1,0.273199,2.533333,wide
2,0.312066,2.181818,wide
3,0.310704,2.130435,wide
4,0.277127,2.526316,wide
...,...,...,...
8709,0.012524,20.000000,closed
8710,0.024852,20.000000,closed
8711,0.018644,20.000000,closed
8712,0.064209,9.666667,closed


In [84]:
df.to_json('data.json', index=False)